## Dependencias

In [40]:
import numpy as np
import pandas as pd 
import os 
from glob import glob 
import json
from sqlalchemy import create_engine
from sqlalchemy.types import BIGINT,VARCHAR,FLOAT,DATE
from multiprocessing import Pool

import yfinance as yf


## Auxiliares 

In [5]:
def make_pool(func, params, threads):
    """
    Ejecuta de forma simultánea múltiples llamadas a una función
    :param func: function, objeto función a paralelizar
    :param params: iterable, parámetros de evaluación paralela
    :param threads: int, número de hilos de multiprocesamiento
    :return: list, resultado de la ejecución paralela agrupada en una lista
    """
    pool = Pool(threads)
    data = pool.starmap(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]


## Datos

In [26]:
snp = pd.read_parquet('s&p100.parquet')
snp.shape
snp.value_counts('Sector',normalize=True)

Sector
Information Technology    0.168317
Financials                0.148515
Health Care               0.138614
Industrials               0.118812
Consumer Discretionary    0.108911
Consumer Staples          0.108911
Communication Services    0.099010
Utilities                 0.039604
Energy                    0.029703
Materials                 0.019802
Real Estate               0.019802
Name: proportion, dtype: float64

In [43]:
def downloadStock(stock:str)->pd.DataFrame:
    """ Descarga los datos de una acción de Yahoo Finance

    Args:
        stock (str): Ticker de la acción

    Returns:
        pd.DataFrame: DataFrame con los datos de la acción
    """    
    df = yf.download(stock, period="1y")
    df.reset_index(inplace=True)
    df.insert(0,'stock',stock)
    df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d').dt.date
    df.columns = ['stock','date','open','high','low','close','adj_close','volume']
    df.drop('adj_close',axis=1,inplace=True)
    for c in df.columns:
        if c not in ['stock','date']:
            df[c] = pd.to_numeric(df[c],errors='coerce')
    return df

In [51]:
def cargarDatos(stock:str):
    try:
        print(f"Cargando acción {stock}...")
        creds = json.load(open('../../creds/mysql.json','r'))
        cnx = create_engine(f'mysql+pymysql://{creds["u"]}:{creds["p"]}@{creds["h"]}/{creds["d"]}').connect()

        data = downloadStock(stock)
        data = data.merge(snp,left_on='stock',right_on='Symbol',how='left').drop('Symbol',axis=1)
        data.to_sql('stocks',
                    cnx,
                    if_exists='append',
                    index=False,
                    dtype=dict(zip(data.columns,
                    [VARCHAR(5),DATE]+[FLOAT]*4+[BIGINT]+[VARCHAR(50)]*2))
            )
        cnx.close()
        return True
    except Exception as e:
        print(e)
        return False

In [55]:
results = make_pool(cargarDatos,[(s,) for s in snp.Symbol],50)

Cargando acción ADBE...Cargando acción ABT...Cargando acción ABBV...Cargando acción ACN...Cargando acción AMGN...Cargando acción AMT...Cargando acción AVGO...
Cargando acción AMZN...Cargando acción AXP...Cargando acción AAPL...

Cargando acción BA...
Cargando acción BAC...


Cargando acción BK...

Cargando acción BMY...


Cargando acción BKNG...Cargando acción BLK...Cargando acción CAT...Cargando acción C...Cargando acción CL...
Cargando acción BRK.B...


Cargando acción CMCSA...Cargando acción COST...Cargando acción COP...Cargando acción COF...Cargando acción CHTR...
Cargando acción CVS...
Cargando acción CVX...Cargando acción CSCO...

Cargando acción CRM...



Cargando acción DIS...Cargando acción DOW...
Cargando acción DHR...
Cargando acción DUK...Cargando acción EXC...
Cargando acción FDX...Cargando acción GD...Cargando acción EMR...
Cargando acción F...





Cargando acción GM...Cargando acción GILD...
Cargando acción GE...
Cargando acción GOOG...Cargando acción HD...Cargando acci


1 Failed download:
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Cargando acción KO...
Cargando acción LIN...
Cargando acción LLY...
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%

In [56]:
sum(results)

101

In [59]:
df = pd.concat(map(downloadStock,snp.Symbol),ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [60]:
df.shape

(25100, 7)

In [62]:
df.to_clipboard(index=False)